# Base Code

In [2]:
from pydantic import (BaseModel, Field, ValidationError, validator)
import pandas as pd
import numpy as np
from tkinter import *
from tkinter import filedialog
import openpyxl
import pyodbc
from typing import Optional,List
import sys
import csv

def browse_files():
       global filename
       filename = filedialog.askopenfilename(initialdir = "/Downloads", title = "Selecciona un archivo de MDM:", filetypes = ((".csv Files","*.csv*"),("all files","*.*")))
       if filename:
              l1 = Label(window, text = "File path: " + filename).pack()
       else:
              print('No seleccionaste ningún archivo.')
       window.destroy()
def export_file():

    export_file_path = filedialog.asksaveasfilename(defaultextension = '.xlsx',initialdir = "/Desktop", title = "Guardar archivo como:", filetypes = (("Excel Files","*.xlsx*"),("CSV Files","*.csv*"),("all files","*.*")))
    data.to_excel(export_file_path, index = False, header=True)
    window2.destroy()
def load_items(df):  
       raw_values = list(df.values)
       headers = list(df.columns)
       dict_raw_items = [dict(zip(headers,row)) for row in raw_values]

       items: List[Item] = [Item(**item) for item in dict_raw_items]
              #items = [Item.parse_obj(sku) for sku in dict_raw_items]
       return items
def db_connect() -> pd.DataFrame:
       print('Connecting to Aurora...')
       try:
              conn = pyodbc.connect(Driver='{SQL Server}',
                                   Server='pr-aurora1-rs01',
                                   Database='db_product_integrity',
                                   Trusted_Connection='yes')
              print('Connected successfully to Aurora')
       except:
              raise Exception ('Connection to Aurora failed. Ending execution.')
       sql = ("""
              SELECT DISTINCT 
              CAST(A.sku AS VARCHAR) + '|' +  CAST(V.AP_REF AS VARCHAR) + '|' +  A.vendor_id AS [key2]
              , CAST(A.sku AS VARCHAR) + '|' +  CAST(V.AP_REF AS VARCHAR)  [key]
              , A.sku
              , V.AP_REF
              , A.vendor_id
              , A.warehouses, ISNULL(B.rank, 999) VENDOR_RANK
              , CASE WHEN A.vendor_id != A.core_return_vendor OR A.vendor_id != A.recall_vendor OR A.vendor_id != A.warranty_vendor THEN 'Yes' ELSE 'No' END [Diff Contracts]
              , LEN(A.warehouses) L
              FROM db_product_integrity.prod.tb_sku_warehouses A WITH(NOLOCK)
              LEFT JOIN db_product_integrity.prod.tb_az_vendors V WITH(NOLOCK)
              ON A.vendor_id = V.VENDOR 
              LEFT JOIN db_product_integrity.prod.tb_item_vendor_rank B WITH(NOLOCK)
              ON A.sku = B.item
                     AND A.vendor_id = b.vendor_id
              WHERE  
              SKU IN({})
              ORDER BY A.sku, VENDOR_RANK, L DESC""").format(skus)
       query = pd.read_sql(sql,conn)
       print('Query run successful. Closing connection...')
       conn.close()
       print('Closed connection to Aurora successfully')
       return query
def colclass_parser(mdm):
       cols_orig = list(mdm.columns)
       cols = list(mdm.columns)
       for i in range(len(cols)):
              cols[i] = cols[i].lower()
              cols[i] = cols[i].replace(' ','_')

       cols_list = list(zip(cols,cols_orig))
       for mac in cols_list:
              print(f"'{mac[0]}': str = Field(alias = '{mac[1]}')")

class Item(BaseModel):

       action: str = Field(alias = 'Action', default=None)
       type: str = Field(alias = 'Type')
       id: str = Field(alias = 'ID')
       name: str = Field(alias = 'Name', default='_EMPTY')
       parts_classification: str = Field(alias = 'Parts Classification')
       brand_aaiaid: str = Field(alias = 'Brand AAIAID')
       part_number: str = Field(alias = 'Part Number')
       subbrand_aaiaid: str = Field(alias = 'SubBrand AAIAID')
       vendor_ownership_id: str = Field(alias = 'Vendor Ownership ID')
       apref: Optional[int] = Field(default = None, alias = 'APREF')
       buyer_group: Optional[str] = Field(default = None, alias = 'Buyer Group')
       stock_keeping_unit: Optional[int] = Field(default = None, alias = 'Stock Keeping Unit')
       vendor_dcs_action: Optional[str] = Field(default = None, alias = 'Vendor DCs.Action')
       vendor_dcs_dcs: Optional[str] = Field(default = None, alias = "Vendor DCs.DC's")
       vendor_dcs_pov_id: Optional[str] = Field(default = None, alias = 'Vendor DCs.POV ID')
       product_integrity_reference: Optional[str] = Field(default = None, alias = 'Product Integrity Reference')
       marketing_flag: Optional[str] = Field(default = None, alias = 'Marketing Flag')
       core_vendor_id: Optional[str] = Field(default = None, alias = 'Core Vendor Id')
       recall_vendor_id: Optional[str] = Field(default = None, alias = 'Recall Vendor Id')
       warranty_vendor_id: Optional[str] = Field(default = None, alias = 'Warranty Vendor Id')
       autozone_part_number: Optional[str] = Field(default = None, alias = 'AutoZone Part number')
       line_code: Optional[str] = Field(default = None, alias = 'Line Code')
       alternate_part_number: Optional[str] = Field(default = None, alias = 'Alternate Part Number')
       alternate_line_code: Optional[str] = Field(default = None, alias = 'Alternate Line Code')
       major_department: Optional[int] = Field(default = None, alias = 'Major Department')
       minor_department: Optional[int] = Field(default = None, alias = 'Minor Department')
       quantity_per_application: Optional[int] = Field(default = None, alias = 'Quantity per Application')
       quantity_per_application_uom: Optional[str] = Field(default = None, alias = 'Quantity per Application.UOM')
       quantity_per_application_qualifier: Optional[str] = Field(default = None, alias = 'Quantity per Application Qualifier')
       store_order_pack: Optional[str] = Field(default = None, alias = 'Store Order Pack')
       uom_for_edi_ordering: Optional[str] = Field(default = None, alias = 'UOM for EDI Ordering')
       store_credit_flag: Optional[str] = Field(default = None, alias = 'Store Credit Flag')
       oil_gal: Optional[int] = Field(default = None, alias = 'Oil gal')
       employee_discount: Optional[str] = Field(default = None, alias = 'Employee Discount')
       warranty: Optional[str] = Field(default = None, alias = 'Warranty')
       warranty_months: Optional[str] = Field(default = None, alias = 'Warranty Months')
       quantity_force_flag: Optional[str] = Field(default = None, alias = 'Quantity Force Flag')
       custom_id: Optional[str] = Field(default = None, alias = 'Custom Id')
       schedule_b: Optional[str] = Field(default = None, alias = 'Harmonized Tariff Code (Schedule B)')
       michigan_flag: Optional[str] = Field(default = None, alias = 'Michigan Flag')
       country_of_origin: Optional[str] = Field(default = None, alias = 'Country of Origin (Primary)')
       item_level_gtin: Optional[str] = Field(default = None, alias = 'Item-Level GTIN')
       item_level_gtin_qualifier: Optional[str] = Field(default = None, alias = 'Item-Level GTIN Qualifier')
       dimensions_each_action: Optional[str] = Field(default = None, alias = 'Dimensions - Each.Action')
       dimensions_each_merchandising_height: Optional[str] = Field(default = None, alias = 'Dimensions - Each.Merchandising Height')
       dimensions_each_merchandising_length: Optional[str] = Field(default = None, alias = 'Dimensions - Each.Merchandising Length')
       dimensions_each_merchandising_width: Optional[str] = Field(default = None, alias = 'Dimensions - Each.Merchandising Width')
       dimensions_each_shipping_height: Optional[str] = Field(default = None, alias = 'Dimensions - Each.Shipping Height')
       dimensions_each_shipping_length: Optional[str] = Field(default = None, alias = 'Dimensions - Each.Shipping Length')
       dimensions_each_shipping_width: Optional[str] = Field(default = None, alias = 'Dimensions - Each.Shipping Width')
       dimensions_each_uom: Optional[str] = Field(default = None, alias = 'Dimensions - Each.UOM')
       package_uom_each: Optional[str] = Field(default = None, alias = 'Package UOM - Each')
       package_level_gtin_each: Optional[str] = Field(default = None, alias = 'Package Level GTIN - Each')
       quantity_of_eaches_in_package_each: Optional[str] = Field(default = None, alias = 'Quantity of Eaches in Package - Each')
       weights_each_action: Optional[str] = Field(default = None, alias = 'Weights - Each.Action')
       weights_each_dimensional_weight: Optional[str] = Field(default = None, alias = 'Weights - Each.Dimensional Weight')
       weights_each_uom: Optional[str] = Field(default = None, alias = 'Weights - Each.UOM')
       weights_each_weight: Optional[str] = Field(default = None, alias = 'Weights - Each.Weight')
       dimensions_case_action: Optional[str] = Field(default = None, alias = 'Dimensions - Case.Action')
       dimensions_case_merchandising_height: Optional[str] = Field(default = None, alias = 'Dimensions - Case.Merchandising Height')
       dimensions_case_merchandising_length: Optional[str] = Field(default = None, alias = 'Dimensions - Case.Merchandising Length')
       dimensions_case_merchandising_width: Optional[str] = Field(default = None, alias = 'Dimensions - Case.Merchandising Width')
       dimensions_case_shipping_height: Optional[str] = Field(default = None, alias = 'Dimensions - Case.Shipping Height')
       dimensions_case_shipping_length: Optional[str] = Field(default = None, alias = 'Dimensions - Case.Shipping Length')
       dimensions_case_shipping_width: Optional[str] = Field(default = None, alias = 'Dimensions - Case.Shipping Width')
       dimensions_case_uom: Optional[str] = Field(default = None, alias = 'Dimensions - Case.UOM')
       package_uom_case: Optional[str] = Field(default = None, alias = 'Package UOM - Case')
       package_level_gtin_case: Optional[str] = Field(default = None, alias = 'Package Level GTIN - Case')
       quantity_of_eaches_in_package_case: Optional[str] = Field(default = None, alias = 'Quantity of Eaches in Package - Case')
       weights_case_action: Optional[str] = Field(default = None, alias = 'Weights - Case.Action')
       weights_case_dimensional_weight: Optional[str] = Field(default = None, alias = 'Weights - Case.Dimensional Weight')
       weights_case_weight: Optional[str] = Field(default = None, alias = 'Weights - Case.Weight')
       weights_case_weight_uom: Optional[str] = Field(default = None, alias = 'Weights - Case.Weight UOM')
       dimensions_inner_pack_action: Optional[str] = Field(default = None, alias = 'Dimensions - Inner Pack.Action')
       dimensions_inner_pack_merchandising_height: Optional[str] = Field(default = None, alias = 'Dimensions - Inner Pack.Merchandising Height')
       dimensions_inner_pack_merchandising_length: Optional[str] = Field(default = None, alias = 'Dimensions - Inner Pack.Merchandising Length')
       dimensions_inner_pack_merchandising_width: Optional[str] = Field(default = None, alias = 'Dimensions - Inner Pack.Merchandising Width')
       dimensions_inner_pack_shipping_height: Optional[str] = Field(default = None, alias = 'Dimensions - Inner Pack.Shipping Height')
       dimensions_inner_pack_shipping_length: Optional[str] = Field(default = None, alias = 'Dimensions - Inner Pack.Shipping Length')
       dimensions_inner_pack_shipping_width: Optional[str] = Field(default = None, alias = 'Dimensions - Inner Pack.Shipping Width')
       dimensions_inner_pack_uom: Optional[str] = Field(default = None, alias = 'Dimensions - Inner Pack.UOM')
       package_uom_inner_pack: Optional[str] = Field(default = None, alias = 'Package UOM - Inner Pack')
       package_level_gtin_inner_pack: Optional[str] = Field(default = None, alias = 'Package Level GTIN - Inner Pack')
       quantity_of_eaches_in_package_inner_pack: Optional[str] = Field(default = None, alias = 'Quantity of Eaches in Package - Inner Pack')
       weights_inner_pack_action: Optional[str] = Field(default = None, alias = 'Weights - Inner Pack.Action')
       weights_inner_pack_dimensional_weight: Optional[str] = Field(default = None, alias = 'Weights - Inner Pack.Dimensional Weight')
       weights_inner_pack_uom: Optional[str] = Field(default = None, alias = 'Weights - Inner Pack.UOM')
       weights_inner_pack_weight: Optional[str] = Field(default = None, alias = 'Weights - Inner Pack.Weight')
       ship_round_quantity: Optional[str] = Field(default = None, alias = 'Ship Round Quantity')
       quantity_per_case: Optional[str] = Field(default = None, alias = 'Quantity per Case')
       pallet_layer_maximum: Optional[str] = Field(default = None, alias = 'Pallet Layer Maximum')
       maximum_cases_per_pallet_layer: Optional[str] = Field(default = None, alias = 'Maximum Cases per Pallet Layer')
       pricing_action: Optional[str] = Field(default = None, alias = 'Pricing.Action')
       pricing_currency_code: Optional[str] = Field(default = None, alias = 'Pricing.Currency Code')
       pricing_expiration_date: Optional[str] = Field(default = None, alias = 'Pricing.Expiration Date')
       pricing_price: Optional[float] = Field(default = None, alias = 'Pricing.Price')
       pricing_price_uom: Optional[str] = Field(default = None, alias = 'Pricing.Price.UOM')
       pricing_price_break_quantity: Optional[str] = Field(default = None, alias = 'Pricing.Price Break Quantity')
       pricing_price_break_quantity_uom: Optional[str] = Field(default = None, alias = 'Pricing.Price Break Quantity.UOM')
       pricing_price_break_quantity_uom2: Optional[str] = Field(default = None, alias = 'Pricing.Price Break Quantity UOM')
       pricing_price_sheet_level_effective_date: Optional[str] = Field(default = None, alias = 'Pricing.Price Sheet Level Effective Date')
       pricing_price_sheet_number: Optional[str] = Field(default = None, alias = 'Pricing.Price Sheet Number')
       pricing_price_type: Optional[str] = Field(default = None, alias = 'Pricing.Price Type')
       pricing_price_type_description: Optional[str] = Field(default = None, alias = 'Pricing.Price Type Description')
       pricing_price_uom: Optional[str] = Field(default = None, alias = 'Pricing.Price UOM')
       retail_cost: Optional[float] = Field(default = None, alias = 'Retail Cost')
       core_retail_price: Optional[float] = Field(default = None, alias = 'Core Retail Price')
       product_description_long: Optional[str] = Field(default = None, alias = 'Product Description – Long')
       assign_az_upc: Optional[str] = Field(default = None, alias = 'Assign AZ UPC')
       assign_sku: Optional[str] = Field(default = None, alias = 'Assign SKU')
       supply_chain_analyst: Optional[str] = Field(default = None, alias = 'Supply Chain Analyst')
       contains_electronic_components: Optional[str] = Field(default = None, alias = 'Contains Electronic Components?')
       does_it_require_sds: Optional[str] = Field(default = None, alias = 'Does it require SDS?')
       is_item_a_chemical: Optional[str] = Field(default = None, alias = 'Is item a chemical?')
       is_or_contains_a_bulb: Optional[str] = Field(default = None, alias = 'Is or contains a bulb?')
       is_or_contains_a_battery: Optional[str] = Field(default = None, alias = 'Is or contains a battery?')

       @validator('vendor_ownership_id','apref')
       def check_apref(cls,v):
              if 'apref'in 'vendor_ownership_id':
                     pass
              else:
                     


# Main

window = Tk()
window.title("Data Governance")
window.geometry("200x100")
label_file_explorer = Label(window, text = "Data Governance", width = 100, height = 4).pack()
button_explore = Button(window, text = "Buscar archivo", command = browse_files).pack()
window.mainloop()

print('Reading MDM file...')
try:
        mdm = pd.read_csv(f'{filename}', index_col = False,na_filter= False)
        skus_list = list(mdm['Stock Keeping Unit'])
        skus_list_string = str(skus_list)
        skus = skus_list_string[1:-1]
        print('MDM file read successfully')
except:
        raise Exception('Error at reading MDM file. Please check file and try again. Finishing execution.')

#query = db_connect()
items = load_items(mdm)


Reading MDM file...
MDM file read successfully


In [4]:
from pydantic import BaseModel, ValidationError, validator


class UserModel(BaseModel):
    name: str
    username: str
    password1: str
    password2: str

    @validator('name')
    def name_must_contain_space(cls, v):
        if ' ' not in v:
            raise ValueError('must contain a space')
        return v.title()

    @validator('password2')
    def passwords_match(cls, v, values, **kwargs):
        if 'password1' in values and v != values['password1']:
            raise ValueError('passwords do not match')
        return v

    @validator('username')
    def username_alphanumeric(cls, v):
        assert v.isalnum(), 'must be alphanumeric'
        return v


user = UserModel(
    name='samuel colvin',
    username='scolvin',
    password1='zxcvbn',
    password2='zxcvbn',
)
print(user)
#> name='Samuel Colvin' username='scolvin' password1='zxcvbn' password2='zxcvbn'

try:
    UserModel(
        name='samuel',
        username='scolvin',
        password1='zxcvbn',
        password2='zxcvbn2',
    )
except ValidationError as e:
    print(e)
    """
    2 validation errors for UserModel
    name
      must contain a space (type=value_error)
    password2
      passwords do not match (type=value_error)
    """

name='Samuel Colvin' username='scolvin' password1='zxcvbn' password2='zxcvbn'
2 validation errors for UserModel
name
  must contain a space (type=value_error)
password2
  passwords do not match (type=value_error)


## MDM Class creation


In [5]:
from pydantic import BaseModel
import pandas as pd

class SomeModel(BaseModel):
    col1: str
    col2: str

data = [SomeModel(**{'col1': 'foo', 'col2': 'bar'})] * 10
pd.DataFrame(data)

,0,1
0,"(col1, foo)","(col2, bar)"
1,"(col1, foo)","(col2, bar)"
2,"(col1, foo)","(col2, bar)"
3,"(col1, foo)","(col2, bar)"
4,"(col1, foo)","(col2, bar)"
5,"(col1, foo)","(col2, bar)"
6,"(col1, foo)","(col2, bar)"
7,"(col1, foo)","(col2, bar)"
8,"(col1, foo)","(col2, bar)"
9,"(col1, foo)","(col2, bar)"


In [ ]:
class Item(BaseModel):

    action: str = Field(alias='Action')
    type: str = Field(alias='Type')
    id: str = Field(alias='ID')
    name: str = Field(alias='Name')
    parts_classification: str = Field(alias='Parts Classification')
    brand_aaiaid: str = Field(alias='Brand AAIAID')
    part_number: str = Field(alias='Part Number')
    subbrand_aaiaid: str = Field(alias='SubBrand AAIAID')
    vendor_ownership_id: str = Field(alias='Vendor Ownership ID')
    apref: str = Field(alias='APREF')
    buyer_group: str = Field(alias='Buyer Group')
    stock_keeping_unit: str = Field(alias='Stock Keeping Unit')
    vendor_dcs_action: str = Field(alias='Vendor DCs.Action')
    vendor_dcs_dcs: str = Field(alias="Vendor DCs.DC's")
    vendor_dcs_pov_id: str = Field(alias='Vendor DCs.POV ID')
    product_integrity_reference: str = Field(alias='Product Integrity Reference')
    marketing_flag: str = Field(alias='Marketing Flag')
    core_vendor_id: str = Field(alias='Core Vendor Id')
    recall_vendor_id: str = Field(alias='Recall Vendor Id')
    warranty_vendor_id: str = Field(alias='Warranty Vendor Id')
    autozone_part_number: str = Field(alias='AutoZone Part number')
    line_code: str = Field(alias='Line Code')
    alternate_part_number: str = Field(alias='Alternate Part Number')
    alternate_line_code: str = Field(alias='Alternate Line Code')
    major_department: str = Field(alias='Major Department')
    minor_department: str = Field(alias='Minor Department')
    quantity_per_application: str = Field(alias='Quantity per Application')
    quantity_per_application_uom: str = Field(alias='Quantity per Application.UOM')
    quantity_per_application_qualifier: str = Field(alias='Quantity per Application Qualifier')
    store_order_pack: str = Field(alias='Store Order Pack')
    uom_for_edi_ordering: str = Field(alias='UOM for EDI Ordering')
    store_credit_flag: str = Field(alias='Store Credit Flag')
    oil_gal: str = Field(alias='Oil gal')
    employee_discount: str = Field(alias='Employee Discount')
    warranty: str = Field(alias='Warranty')
    warranty_months: str = Field(alias='Warranty Months')
    quantity_force_flag: str = Field(alias='Quantity Force Flag')
    custom_id: str = Field(alias='Custom Id')
    schedule_b: str = Field(alias='Harmonized Tariff Code (Schedule B)')
    michigan_flag: str = Field(alias='Michigan Flag')
    country_of_origin: str = Field(alias='Country of Origin (Primary)')
    item_level_gtin: str = Field(alias='Item-Level GTIN')
    item_level_gtin_qualifier: str = Field(alias='Item-Level GTIN Qualifier')
    dimensions_each_action: str = Field(alias='Dimensions - Each.Action')
    dimensions_each_merchandising_height: str = Field(alias='Dimensions - Each.Merchandising Height')
    dimensions_each_merchandising_length: str = Field(alias='Dimensions - Each.Merchandising Length')
    dimensions_each_merchandising_width: str = Field(alias='Dimensions - Each.Merchandising Width')
    dimensions_each_shipping_height: str = Field(alias='Dimensions - Each.Shipping Height')
    dimensions_each_shipping_length: str = Field(alias='Dimensions - Each.Shipping Length')
    dimensions_each_shipping_width: str = Field(alias='Dimensions - Each.Shipping Width')
    dimensions_each_uom: str = Field(alias='Dimensions - Each.UOM')
    package_uom_each: str = Field(alias='Package UOM - Each')
    package_level_gtin_each: str = Field(alias='Package Level GTIN - Each')
    quantity_of_eaches_in_package_each: str = Field(alias='Quantity of Eaches in Package - Each')
    weights_each_action: str = Field(alias='Weights - Each.Action')
    weights_each_dimensional_weight: str = Field(alias='Weights - Each.Dimensional Weight')
    weights_each_uom: str = Field(alias='Weights - Each.UOM')
    weights_each_weight: str = Field(alias='Weights - Each.Weight')
    dimensions_case_action: str = Field(alias='Dimensions - Case.Action')
    dimensions_case_merchandising_height: str = Field(alias='Dimensions - Case.Merchandising Height')
    dimensions_case_merchandising_length: str = Field(alias='Dimensions - Case.Merchandising Length')
    dimensions_case_merchandising_width: str = Field(alias='Dimensions - Case.Merchandising Width')
    dimensions_case_shipping_height: str = Field(alias='Dimensions - Case.Shipping Height')
    dimensions_case_shipping_length: str = Field(alias='Dimensions - Case.Shipping Length')
    dimensions_case_shipping_width: str = Field(alias='Dimensions - Case.Shipping Width')
    dimensions_case_uom: str = Field(alias='Dimensions - Case.UOM')
    package_uom_case: str = Field(alias='Package UOM - Case')
    package_level_gtin_case: str = Field(alias='Package Level GTIN - Case')
    quantity_of_eaches_in_package_case: str = Field(alias='Quantity of Eaches in Package - Case')
    weights_case_action: str = Field(alias='Weights - Case.Action')
    weights_case_dimensional_weight: str = Field(alias='Weights - Case.Dimensional Weight')
    weights_case_weight: str = Field(alias='Weights - Case.Weight')
    weights_case_weight_uom: str = Field(alias='Weights - Case.Weight UOM')
    dimensions_inner_pack_action: str = Field(alias='Dimensions - Inner Pack.Action')
    dimensions_inner_pack_merchandising_height: str = Field(alias='Dimensions - Inner Pack.Merchandising Height')
    dimensions_inner_pack_merchandising_length: str = Field(alias='Dimensions - Inner Pack.Merchandising Length')
    dimensions_inner_pack_merchandising_width: str = Field(alias='Dimensions - Inner Pack.Merchandising Width')
    dimensions_inner_pack_shipping_height: str = Field(alias='Dimensions - Inner Pack.Shipping Height')
    dimensions_inner_pack_shipping_length: str = Field(alias='Dimensions - Inner Pack.Shipping Length')
    dimensions_inner_pack_shipping_width: str = Field(alias='Dimensions - Inner Pack.Shipping Width')
    dimensions_inner_pack_uom: str = Field(alias='Dimensions - Inner Pack.UOM')
    package_uom_inner_pack: str = Field(alias='Package UOM - Inner Pack')
    package_level_gtin_inner_pack: str = Field(alias='Package Level GTIN - Inner Pack')
    quantity_of_eaches_in_package_inner_pack: str = Field(alias='Quantity of Eaches in Package - Inner Pack')
    weights_inner_pack_action: str = Field(alias='Weights - Inner Pack.Action')
    weights_inner_pack_dimensional_weight: str = Field(alias='Weights - Inner Pack.Dimensional Weight')
    weights_inner_pack_uom: str = Field(alias='Weights - Inner Pack.UOM')
    weights_inner_pack_weight: str = Field(alias='Weights - Inner Pack.Weight')
    ship_round_quantity: str = Field(alias='Ship Round Quantity')
    quantity_per_case: str = Field(alias='Quantity per Case')
    pallet_layer_maximum: str = Field(alias='Pallet Layer Maximum')
    maximum_cases_per_pallet_layer: str = Field(alias='Maximum Cases per Pallet Layer')
    pricing_action: str = Field(alias='Pricing.Action')
    pricing_currency_code: str = Field(alias='Pricing.Currency Code')
    pricing_expiration_date: str = Field(alias='Pricing.Expiration Date')
    pricing_price: str = Field(alias='Pricing.Price')
    pricing_price_uom: str = Field(alias='Pricing.Price.UOM')
    pricing_price_break_quantity: str = Field(alias='Pricing.Price Break Quantity')
    pricing_price_break_quantity_uom: str = Field(alias='Pricing.Price Break Quantity.UOM')
    pricing_price_break_quantity_uom2: str = Field(alias='Pricing.Price Break Quantity UOM')
    pricing_price_sheet_level_effective_date: str = Field(alias='Pricing.Price Sheet Level Effective Date')
    pricing_price_sheet_number: str = Field(alias='Pricing.Price Sheet Number')
    pricing_price_type: str = Field(alias='Pricing.Price Type')
    pricing_price_type_description: str = Field(alias='Pricing.Price Type Description')
    pricing_price_uom: str = Field(alias='Pricing.Price UOM')
    retail_cost: str = Field(alias='Retail Cost')
    core_retail_price: str = Field(alias='Core Retail Price')
    product_description_long: str = Field(alias='Product Description – Long')
    assign_az_upc: str = Field(alias='Assign AZ UPC')
    assign_sku: str = Field(alias='Assign SKU')
    supply_chain_analyst: str = Field(alias='Supply Chain Analyst')
    contains_electronic_components: str = Field(alias='Contains Electronic Components?')
    does_it_require_sds: str = Field(alias='Does it require SDS?')
    is_item_a_chemical: str = Field(alias='Is item a chemical?')
    is_or_contains_a_bulb: str = Field(alias='Is or contains a bulb?')
    is_or_contains_a_battery: str = Field(alias='Is or contains a battery?')

    
items = [Item.parse_obj(sku) for sku in dict_raw_items]

In [ ]:
class Item:
    def __init__(self,id,sku,void,pov,dcs):
        self.id = id 
        self.sku = sku
        self.void = void
        self.pov = pov
        self.dcs = dcs
test = Item('0acb4b8d-3a96-45b5-880e-c4add8902517',1106491,'26906-USA','45195','91')
print(test.sku)